<a href="https://colab.research.google.com/github/raghavmehta1/ImageClassification/blob/main/MFO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import optunity
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load your dataset
file_path = 'dataset.csv'
data = pd.read_csv(file_path)

# Encode categorical labels in the target column
label_encoder = LabelEncoder()
data['Class/ASD Traits '] = label_encoder.fit_transform(data['Class/ASD Traits '])

# Extract features (X) and target variable (y)
numeric_features = data.select_dtypes(include=['float64', 'int64']).columns
X = data[numeric_features].drop(columns=['Case_No', 'Class/ASD Traits '])
y = data['Class/ASD Traits ']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function with two fidelity levels
def objective_function_low_fidelity(n_estimators_low):
    # Train a Random Forest classifier with the given hyperparameters
    model = RandomForestClassifier(n_estimators=int(n_estimators_low), random_state=42)
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Evaluate the model using accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Since Optunity minimizes, return the negative accuracy
    return -accuracy

def objective_function_high_fidelity(n_estimators_high):
    # Train a Random Forest classifier with the given hyperparameters
    model = RandomForestClassifier(n_estimators=int(n_estimators_high), random_state=42)
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Evaluate the model using accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Since Optunity minimizes, return the negative accuracy
    return -accuracy

# Set up the search space for hyperparameters at each fidelity level
search_space_low_fidelity = {'n_estimators_low': [10, 50]}
search_space_high_fidelity = {'n_estimators_high': [50, 200]}

# Use Optunity to optimize hyperparameters at each fidelity level
optimal_params_low_fidelity, _, _ = optunity.minimize(objective_function_low_fidelity, num_evals=10, **search_space_low_fidelity)
optimal_params_high_fidelity, _, _ = optunity.minimize(objective_function_high_fidelity, num_evals=10, **search_space_high_fidelity)

# Extract the optimized hyperparameters at each fidelity level
best_n_estimators_low = optimal_params_low_fidelity['n_estimators_low']
best_n_estimators_high = optimal_params_high_fidelity['n_estimators_high']

# Train the final model with the best hyperparameters
final_model_low_fidelity = RandomForestClassifier(n_estimators=int(best_n_estimators_low), random_state=42)
final_model_high_fidelity = RandomForestClassifier(n_estimators=int(best_n_estimators_high), random_state=42)

final_model_low_fidelity.fit(X_train, y_train)
final_model_high_fidelity.fit(X_train, y_train)

# Make predictions on the test set
y_final_pred_low_fidelity = final_model_low_fidelity.predict(X_test)
y_final_pred_high_fidelity = final_model_high_fidelity.predict(X_test)

# Evaluate the final models
final_accuracy_low_fidelity = accuracy_score(y_test, y_final_pred_low_fidelity)
final_accuracy_high_fidelity = accuracy_score(y_test, y_final_pred_high_fidelity)

print(f"Final Accuracy (Low Fidelity): {final_accuracy_low_fidelity:.4f}")
print(f"Final Accuracy (High Fidelity): {final_accuracy_high_fidelity:.4f}")


Final Accuracy (Low Fidelity): 1.0000
Final Accuracy (High Fidelity): 1.0000


In [17]:
!pip install --upgrade optunity


In [1]:
pip install optunity


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for optunity: filename=Optunity-1.1.1-py3-none-any.whl size=72029 sha256=8f412de07f37a3e44c04d47e758cfbdcc7b94bcba753b8aba6c70e511cfda7ba
  Stored in directory: /root/.cache/pip/wheels/83/98/40/b9956c3386a045f035f9edbf894edd04713db92f29fb58a225
Successfully built optunity
